In [1]:
import tensorflow as tf
import os

In [2]:
os.chdir("../")

In [3]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    params_image_size: list
    params_batch_size: int 


In [5]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath) #reading config file
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data = "artifacts/data_ingestion/PetImages",
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        return eval_config

In [6]:
import os
import tensorflow as tf
import time
from deepClassifier.utils import *
from pathlib import Path

class Evaluation:
    def __init__(self, config: EvaluationConfig): #takes input in local config variable with i/p datatype as TrainingConfig
        self.config = config
    
    #creating generator
    def _valid_generator(self):

        datagenerator_kwargs = dict( #it is a dictionary
            rescale = 1./255,
            validation_split = 0.30
        )

        # for image resizing
        dataflow_kwargs = dict( #it is a dictionary
            target_size = self.config.params_image_size [:-1], #channel info of image is not required, hence we take -1, means all rows and coloumns except last.
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs #passing the dict as kwargs to this function. This helps to pass multiple args at once
        )

        # creating validation data generator
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model: #this is static method hence doesn't need self.
        return tf.keras.models.load_model(path)

    def evaluation(self):
        model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator) # evaluate method returns accuracy and metrics information after evaluating the model
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]} #dict with evaluation results. i.e loss and accuracy
        save_json(path = Path("scores.json"), data = scores) #storing in root dir by not specifying full path, so that dvc can track it easily.


In [7]:
#pipeline section
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(config = val_config)
    out = evaluation.evaluation() #running evaluation method from our user defined Evaluation class
    evaluation.save_score()

except Exception as e:
    raise e

Found 7498 images belonging to 2 classes.
  1/469 [..............................] - ETA: 6:54:05 - loss: 0.4104 - accuracy: 0.7500